In [1]:
import dedup_bindings

In [2]:
import polars as pl


df = pl.read_parquet("/home/wayne/kioxia/github/text-dedup/temp_inp_paruqet/data.parquet")[:1000]

In [3]:
def shingle (text: str, k: int):
    text = text.lower()
    shingles = []
    for i in range(len(text) - k + 1):
        shingles.append(text[i:i+k])
    return set(shingles)

In [4]:
rust_impl = df['text'].map_elements(lambda s: dedup_bindings.shingle(s, 3))

/tmp/ipykernel_16611/3254413304.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  rust_impl = df['text'].map_elements(lambda s: dedup_bindings.shingle(s, 3))


In [5]:
py_impl = df['text'].map_elements(lambda s: shingle(s, 3))

/tmp/ipykernel_16611/4050695236.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  py_impl = df['text'].map_elements(lambda s: shingle(s, 3))


In [6]:
vocab = set()
for i in range(len(rust_impl)):
    vocab.update(rust_impl[i])
vocab = tuple(vocab)

In [12]:
one_hot = [1 if x in rust_impl[0] else 0 for x in vocab]
hashex = list(range(len(vocab)))

In [13]:
from random import shuffle
shuffle(hashex)

In [14]:
for i in range(1, 10):
    print(f"{i} -> {hashex.index(i)}")

1 -> 7508
2 -> 3016
3 -> 4345
4 -> 5737
5 -> 5624
6 -> 1706
7 -> 3608
8 -> 6108
9 -> 5680


In [15]:
for i in range(1, len(vocab)+1):
    idx = hashex.index(i)
    sig_val = one_hot[idx]
    print(f"{i} -> {idx} -> {sig_val}")
    if sig_val == 1:
        break

1 -> 7508 -> 0
2 -> 3016 -> 0
3 -> 4345 -> 0
4 -> 5737 -> 0
5 -> 5624 -> 0
6 -> 1706 -> 0
7 -> 3608 -> 0
8 -> 6108 -> 0
9 -> 5680 -> 0
10 -> 5480 -> 0
11 -> 8411 -> 1


In [7]:
import numpy as np
2**61-1

2305843009213693951

In [4]:
from itertools import tee
from typing import List
def ngrams(sequence: List[str], n: int, min_length: int = 5):
    """
    Return the ngrams generated from a sequence of items, as an iterator.

    This is a modified version of nltk.util.ngrams.

    Parameters
    ----------
    sequence : List[Text]
        The sequence of items.
    n : int
        The length of each ngram.
    min_length : int, optional
        The minimum length of each ngram, by default 5

    Returns
    -------
    iterator
        The ngrams.

    Examples
    --------
    >>> list(ngrams(["a", "b", "c", "d"], 2, min_length=1))
    [('a', 'b'), ('b', 'c'), ('c', 'd')]
    >>> list(ngrams(["a", "b", "c", "d"], 2, min_length=5))
    []
    >>> list(ngrams(["a", "b"], 3, min_length=1))
    [('a', 'b')]
    """
    if len(sequence) < min_length:
        return []
    if len(sequence) < n:
        return [tuple(sequence)]
    iterables = tee(iter(sequence), n)
    for i, sub_iterable in enumerate(iterables):
        for _ in range(i):
            next(sub_iterable, None)
    return zip(*iterables)


In [5]:
import hashlib
import struct

def sha1_hash(data: bytes, d: int = 32) -> int:
    """
    Generate a d-bit hash value from the given data.

    Parameters
    ----------
    data : bytes
        The data to be hashed.
    d : int
        The number of bits of the hash value.

    Returns
    -------
    int
        The hash value.

    Examples
    --------
    >>> sha1_hash(b"hello world", 32)
    896314922
    >>> sha1_hash(b"hello world", 64)
    13028719972609469994
    >>> sha1_hash(b"hello world", 128)
    310522945683037930239412421226792791594
    """
    if d == 32:
        return struct.unpack("<I", hashlib.sha1(data, usedforsecurity=False).digest()[:4])[0]
    if d == 64:
        return struct.unpack("<Q", hashlib.sha1(data, usedforsecurity=False).digest()[:8])[0]
    # struct is faster but does not support arbitrary bit lengths
    return int.from_bytes(hashlib.sha1(data, usedforsecurity=False).digest()[: d // 8], byteorder="little")


In [6]:
import re
import numpy as np
NON_ALPHA = re.compile(r"\W", re.UNICODE)
content = "But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of human happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know how to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and pain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, except to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that has no annoying consequences, or one who avoids a pain that produces no resultant pleasure?"

tokens: set[bytes] = {
        bytes(" ".join(t).lower(), "utf-8") for t in ngrams(NON_ALPHA.split(content.lower()), 3, 5)
    }

hashvalues: np.ndarray = np.array([sha1_hash(token) for token in tokens], dtype=np.uint64).reshape(len(tokens), 1)


In [1]:
# Permute the hash values to produce new universal hashes
# Element-wise multiplication with 'hashvalues' and a (non 0 random value) and then adding b
# Then, take modulo 'MODULO_PRIME' and bitwise_and with 'MAX_HASH' to keep only the necessary bits.
max_hash = np.uint32((1 << 32) - 1)
modulo_prime = np.uint64((1 << 61) - 1)
PERMUTATIONS: tuple[np.ndarray, np.ndarray] = (
RNG.randint(
    1, MODULO_PRIME, size=(minhash_args.num_perm,), dtype=DTYPE
),  # a is a multiplier so should not be 0
RNG.randint(0, MODULO_PRIME, size=(minhash_args.num_perm,), dtype=DTYPE),  # b
)

hashvalues = (hashvalues * a + b) % modulo_prime & max_hash

NameError: name 'np' is not defined

In [27]:
rust = {"a complete account", "a man who", "a pain that", "a pleasure that", "a trivial example", "account of the", "actual teachings of", "advantage from it", "again is there", "all this mistaken", "and expound the", "and i will", "and pain can", "and praising pain", "annoying consequences or", "any right to", "anyone who loves", "are extremely painful", "avoids a pain", "avoids pleasure itself", "because it is", "because occasionally circumstances", "because those who", "born and i", "builder of human", "but because occasionally", "but because those", "but i must", "but who has", "can procure him", "chooses to enjoy", "circumstances occur in", "complete account of", "consequences or one", "consequences that are", "denouncing pleasure and", "desires to obtain", "dislikes or avoids", "do not know", "encounter consequences that", "enjoy a pleasure", "ever undertakes laborious", "example which of", "except to obtain", "exercise except to", "explain to you", "explorer of the", "expound the actual", "extremely painful nor", "fault with a", "find fault with", "from it but", "give you a", "great explorer of", "great pleasure to", "happiness no one", "has any right", "has no annoying", "him some great", "how all this", "how to pursue", "human happiness no", "i must explain", "i will give", "idea of denouncing", "in which toil", "is pain but", "is pleasure but", "is there anyone", "it but who", "it is pain", "it is pleasure", "itself because it", "know how to", "laborious physical exercise", "loves or pursues", "man who chooses", "master builder of", "mistaken idea of", "must explain to", "no annoying consequences", "no one rejects", "no resultant pleasure", "nor again is", "not know how", "obtain pain of", "obtain some advantage", "occasionally circumstances occur", "occur in which", "of denouncing pleasure", "of human happiness", "of itself because", "of the great", "of the system", "of the truth", "of us ever", "one rejects dislikes", "one who avoids", "or avoids pleasure", "or desires to", "or one who", "or pursues or", "pain but because", "pain can procure", "pain of itself", "pain that produces", "pain was born", "painful nor again", "physical exercise except", "pleasure and praising", "pleasure but because", "pleasure itself because", "pleasure rationally encounter", "pleasure that has", "pleasure to take", "praising pain was", "procure him some", "produces no resultant", "pursue pleasure rationally", "pursues or desires", "rationally encounter consequences", "rejects dislikes or", "right to find", "some advantage from", "some great pleasure", "system and expound", "take a trivial", "teachings of the", "that are extremely", "that has no", "that produces no", "the actual teachings", "the great explorer", "the master builder", "the system and", "the truth the", "there anyone who", "this mistaken idea", "those who do", "to enjoy a", "to find fault", "to obtain pain", "to obtain some", "to pursue pleasure", "to take a", "to you how", "toil and pain", "trivial example which", "truth the master", "undertakes laborious physical", "us ever undertakes", "was born and", "which of us", "which toil and", "who avoids a", "who chooses to", "who do not", "who has any", "who loves or", "will give you", "with a man", "you a complete", "you how all"}

In [4]:
2**32 - 1

4294967295

In [5]:
import numpy as np
MODULO_PRIME = 2 ** 61 - 1
SEED = 42
DTYPE = np.uint64
RNG = np.random.RandomState(SEED)
SIGNATURE_COLUMN = "__signatures__"

PERMUTATIONS: tuple[np.ndarray, np.ndarray] = (
        RNG.randint(
            1, MODULO_PRIME, size=(200,), dtype=DTYPE
        ),  # a is a multiplier so should not be 0
        RNG.randint(0, MODULO_PRIME, size=(200,), dtype=DTYPE),  # b
)

In [6]:
PERMUTATIONS

(array([2297359619001564596, 1396682528897996047, 1973689801170867272,
        1819927849474927637,  572192888165898362,  571748048327668951,
        1071453510346823115, 2143071682933157237, 1865242737500154728,
        1532418594269339779,  379716980844854581, 1750777661071092056,
        1520798217888136606, 1611122243678701954, 1048235628103308220,
        1077373049702139041, 1000592739665268026,  456675647751657725,
        1050450229337746265,  760543396461257179, 2063321781277379071,
         267364570529679360,  777431531920034492, 2146497176616232719,
        1495477363401685950,  648881935158774718, 1377488174878099308,
         262579006026876148, 1704747971477620217,  856858769658023397,
        1983852346201383559,  839771884757445449, 1199990111390225682,
        1362947483897176665, 1671865959972055054, 1077240956986238970,
        1007446255075745114, 1801732545371167618, 1092656530294298716,
        1201851346935503804, 2251208029941317036, 2216872713715991304,
      